In [5]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

print("Environment ready for data cleaning")


Environment ready for data cleaning


In [7]:
import os

os.listdir("../data/raw")


[' nifty options 5min.csv', '.ipynb_checkpoints', 'nifty futures 5min.csv']

In [9]:
options_df = pd.read_csv(
    r"C:\Users\Janvi\Quant_Trading_Assignment\data\raw\ nifty options 5min.csv"
)

futures_df = pd.read_csv(
   r"C:\Users\Janvi\Quant_Trading_Assignment\data\raw\nifty futures 5min.csv"
)

In [11]:
options_df.head()

,CONTRACT_D,PREVIOUS_S,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,CLOSE_PRIC,SETTLEMENT,NET_CHANGE,OI_NO_CON,TRADED_QUA,TRD_NO_CON,UNDRLNG_ST,NOTIONAL_V,PREMIUM_TR
0,OPTSTK360ONE27-JAN-2026PE1240,104.90,95.5,95.5,86.15,86.15,86.15,18.75,71.0,1000.0,2.0,1149.25,1330825.0,90825.0
1,OPTSTK360ONE24-FEB-2026CE1160,47.00,0.0,0.0,0.00,NaN,52.35,5.35,0.0,0.0,0.0,1149.25,0.0,0.0
2,OPTSTK360ONE30-MAR-2026CE1240,39.35,0.0,0.0,0.00,NaN,43.20,3.85,0.0,0.0,0.0,1149.25,0.0,0.0
3,OPTSTK360ONE24-FEB-2026PE1000,8.30,0.0,0.0,0.00,NaN,6.60,1.70,0.0,0.0,0.0,1149.25,0.0,0.0
4,OPTSTK360ONE27-JAN-2026PE1140,35.40,36.0,36.0,17.40,22.30,22.30,13.10,227.0,156000.0,312.0,1149.25,181740700.0,3900700.0


In [13]:
futures_df.head()

,CONTRACT_D,PREVIOUS_S,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,CLOSE_PRIC,SETTLEMENT,NET_CHANGE,OI_NO_CON,TRADED_QUA,TRD_NO_CON,TRADED_VAL
0,FUTSTK360ONE30-MAR-2026,1149.90,1152.60,1152.6,1152.60,1152.60,1162.30,1.07,5.0,500.0,1.0,5.763000e+05
1,FUTSTK360ONE24-FEB-2026,1131.60,1130.30,1159.5,1130.30,1151.20,1151.20,1.73,366.0,61000.0,122.0,6.981780e+07
2,FUTSTK360ONE27-JAN-2026,1137.20,1128.00,1167.0,1128.00,1155.30,1155.30,1.59,4971.0,1147000.0,2294.0,1.319876e+09
3,FUTSTKAMBUJACEM30-MAR-2026,545.15,543.30,558.4,543.00,556.05,556.05,1.99,156.0,54600.0,52.0,3.019968e+07
4,FUTSTKAMBUJACEM24-FEB-2026,541.80,538.05,554.7,537.55,553.10,553.10,2.08,1652.0,400050.0,381.0,2.197244e+08


In [15]:
print("Options shape:", options_df.shape)
print("Futures shape:", futures_df.shape)


Options shape: (41219, 14)
Futures shape: (636, 12)


In [17]:
options_df.columns

Index(['CONTRACT_D', 'PREVIOUS_S', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRIC', 'SETTLEMENT', 'NET_CHANGE',
       'OI_NO_CON', 'TRADED_QUA', 'TRD_NO_CON', 'UNDRLNG_ST', 'NOTIONAL_V', 'PREMIUM_TR'],
      dtype='object')

In [19]:
futures_df.columns

Index(['CONTRACT_D', 'PREVIOUS_S', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRIC', 'SETTLEMENT', 'NET_CHANGE',
       'OI_NO_CON', 'TRADED_QUA', 'TRD_NO_CON', 'TRADED_VAL'],
      dtype='object')

In [21]:
# Check missing values in options data
options_missing = options_df.isna().sum().sort_values(ascending=False)
options_missing

CLOSE_PRIC    26025
PREVIOUS_S      614
NET_CHANGE      614
CONTRACT_D        0
OPEN_PRICE        0
HIGH_PRICE        0
LOW_PRICE         0
SETTLEMENT        0
OI_NO_CON         0
TRADED_QUA        0
TRD_NO_CON        0
UNDRLNG_ST        0
NOTIONAL_V        0
PREMIUM_TR        0
dtype: int64

In [23]:
# Check missing values in futures data
futures_missing = futures_df.isna().sum().sort_values(ascending=False)
futures_missing

CLOSE_PRIC    9
CONTRACT_D    0
PREVIOUS_S    0
OPEN_PRICE    0
HIGH_PRICE    0
LOW_PRICE     0
SETTLEMENT    0
NET_CHANGE    0
OI_NO_CON     0
TRADED_QUA    0
TRD_NO_CON    0
TRADED_VAL    0
dtype: int64

In [25]:
# Drop options rows where close price is missing
options_df_clean = options_df.dropna(subset=["CLOSE_PRIC"]).reset_index(drop=True)

print("Options before:", options_df.shape)
print("Options after :", options_df_clean.shape)


Options before: (41219, 14)
Options after : (15194, 14)


In [27]:
# Forward fill missing close prices in futures
futures_df_clean = futures_df.sort_index().ffill()

print("Futures before:", futures_df.shape)
print("Futures after :", futures_df_clean.shape)


Futures before: (636, 12)
Futures after : (636, 12)


In [29]:
# Function to remove outliers using IQR
def remove_outliers_iqr(df, cols):
    Q1 = df[cols].quantile(0.25)
    Q3 = df[cols].quantile(0.75)
    IQR = Q3 - Q1

    mask = ~((df[cols] < (Q1 - 1.5 * IQR)) | (df[cols] > (Q3 + 1.5 * IQR))).any(axis=1)
    return df[mask]

price_cols = ["OPEN_PRICE", "HIGH_PRICE", "LOW_PRICE", "CLOSE_PRIC"]

options_before = options_df_clean.shape
options_df_clean = remove_outliers_iqr(options_df_clean, price_cols)
options_after = options_df_clean.shape

print("Options before outliers:", options_before)
print("Options after outliers :", options_after)


Options before outliers: (15194, 14)
Options after outliers : (12890, 14)


In [31]:
futures_before = futures_df_clean.shape
futures_df_clean = remove_outliers_iqr(futures_df_clean, price_cols)
futures_after = futures_df_clean.shape

print("Futures before outliers:", futures_before)
print("Futures after outliers :", futures_after)


Futures before outliers: (636, 12)
Futures after outliers : (555, 12)


In [33]:
print(options_df_clean.columns)
print(futures_df_clean.columns)

Index(['CONTRACT_D', 'PREVIOUS_S', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRIC', 'SETTLEMENT', 'NET_CHANGE',
       'OI_NO_CON', 'TRADED_QUA', 'TRD_NO_CON', 'UNDRLNG_ST', 'NOTIONAL_V', 'PREMIUM_TR'],
      dtype='object')
Index(['CONTRACT_D', 'PREVIOUS_S', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRIC', 'SETTLEMENT', 'NET_CHANGE',
       'OI_NO_CON', 'TRADED_QUA', 'TRD_NO_CON', 'TRADED_VAL'],
      dtype='object')


In [35]:
options_df_clean.head(2)


,CONTRACT_D,PREVIOUS_S,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,CLOSE_PRIC,SETTLEMENT,NET_CHANGE,OI_NO_CON,TRADED_QUA,TRD_NO_CON,UNDRLNG_ST,NOTIONAL_V,PREMIUM_TR
0,OPTSTK360ONE27-JAN-2026PE1240,104.9,95.5,95.5,86.15,86.15,86.15,18.75,71.0,1000.0,2.0,1149.25,1330825.0,90825.0
1,OPTSTK360ONE27-JAN-2026PE1140,35.4,36.0,36.0,17.40,22.30,22.30,13.10,227.0,156000.0,312.0,1149.25,181740700.0,3900700.0


In [37]:
futures_df_clean.head(2)


,CONTRACT_D,PREVIOUS_S,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,CLOSE_PRIC,SETTLEMENT,NET_CHANGE,OI_NO_CON,TRADED_QUA,TRD_NO_CON,TRADED_VAL
0,FUTSTK360ONE30-MAR-2026,1149.9,1152.6,1152.6,1152.6,1152.6,1162.3,1.07,5.0,500.0,1.0,576300.0
1,FUTSTK360ONE24-FEB-2026,1131.6,1130.3,1159.5,1130.3,1151.2,1151.2,1.73,366.0,61000.0,122.0,69817800.0


In [39]:
import re

def parse_contract(contract):
    result = {
        "instrument": None,
        "expiry": None,
        "option_type": None,
        "strike": None
    }
    
    if contract.startswith("OPT"):
        result["instrument"] = "OPTION"
        # Example: OPTSTK3600NE27-JAN-2026PE1240
        match = re.search(r'([A-Z]{3}-\d{4})(CE|PE)(\d+)', contract)
        if match:
            result["expiry"] = match.group(1)
            result["option_type"] = match.group(2)
            result["strike"] = int(match.group(3))
            
    elif contract.startswith("FUT"):
        result["instrument"] = "FUTURE"
        # Example: FUTSTK3600NE30-MAR-2026
        match = re.search(r'([A-Z]{3}-\d{4})', contract)
        if match:
            result["expiry"] = match.group(1)
    
    return pd.Series(result)


In [41]:
options_contract_info = options_df_clean["CONTRACT_D"].apply(parse_contract)
options_df_clean = pd.concat([options_df_clean, options_contract_info], axis=1)

futures_contract_info = futures_df_clean["CONTRACT_D"].apply(parse_contract)
futures_df_clean = pd.concat([futures_df_clean, futures_contract_info], axis=1)

In [42]:
options_df_clean[["CONTRACT_D", "instrument", "expiry", "option_type", "strike"]].head()

,CONTRACT_D,instrument,expiry,option_type,strike
0,OPTSTK360ONE27-JAN-2026PE1240,OPTION,JAN-2026,PE,1240
1,OPTSTK360ONE27-JAN-2026PE1140,OPTION,JAN-2026,PE,1140
2,OPTSTK360ONE24-FEB-2026PE1100,OPTION,FEB-2026,PE,1100
3,OPTSTK360ONE27-JAN-2026CE1200,OPTION,JAN-2026,CE,1200
4,OPTSTK360ONE27-JAN-2026CE1100,OPTION,JAN-2026,CE,1100


In [43]:
futures_df_clean[["CONTRACT_D", "instrument", "expiry"]].head()

,CONTRACT_D,instrument,expiry
0,FUTSTK360ONE30-MAR-2026,FUTURE,MAR-2026
1,FUTSTK360ONE24-FEB-2026,FUTURE,FEB-2026
2,FUTSTK360ONE27-JAN-2026,FUTURE,JAN-2026
3,FUTSTKAMBUJACEM30-MAR-2026,FUTURE,MAR-2026
4,FUTSTKAMBUJACEM24-FEB-2026,FUTURE,FEB-2026


In [44]:
# Identify most liquid futures contract per expiry
futures_df_rollover = (
    futures_df_clean
    .sort_values("TRADED_QUA", ascending=False)
    .groupby("expiry")
    .head(1)
    .reset_index(drop=True)
)

print("Futures before rollover:", futures_df_clean.shape)
print("Futures after rollover:", futures_df_rollover.shape)

Futures before rollover: (555, 16)
Futures after rollover: (3, 16)


In [45]:
futures_df_rollover[["CONTRACT_D", "expiry", "TRADED_QUA"]]

,CONTRACT_D,expiry,TRADED_QUA
0,FUTSTKIDEA27-JAN-2026,JAN-2026,526484850.0
1,FUTSTKIDEA24-FEB-2026,FEB-2026,95061750.0
2,FUTSTKIDEA30-MAR-2026,MAR-2026,19584150.0


In [46]:
futures_df_rollover[["expiry", "SETTLEMENT"]]


,expiry,SETTLEMENT
0,JAN-2026,11.11
1,FEB-2026,11.18
2,MAR-2026,11.25


In [47]:
futures_price_map = (
    futures_df_rollover
    .set_index("expiry")["SETTLEMENT"] * 100
)

futures_price_map

expiry
JAN-2026    1111.0
FEB-2026    1118.0
MAR-2026    1125.0
Name: SETTLEMENT, dtype: float64

In [48]:
options_df_clean["futures_price"] = (
    options_df_clean["expiry"].map(futures_price_map)
)



In [49]:
options_df_clean[["expiry", "strike", "futures_price"]].head()


,expiry,strike,futures_price
0,JAN-2026,1240,1111.0
1,JAN-2026,1140,1111.0
2,FEB-2026,1100,1118.0
3,JAN-2026,1200,1111.0
4,JAN-2026,1100,1111.0


In [50]:
options_df_clean["strike_diff"] = (
    options_df_clean["strike"] - options_df_clean["futures_price"]
).abs()

In [59]:
atm_idx = options_df_clean.groupby("expiry")["strike_diff"].idxmin()

In [62]:
atm_idx.isna().sum()

3

In [65]:
atm_idx = (
    options_df_clean
    .groupby("expiry")["strike_diff"]
    .idxmin()
    .dropna()
)


In [67]:
atm_strikes = (
    options_df_clean
    .loc[atm_idx, ["expiry", "strike", "futures_price"]]
    .sort_values("expiry")
)

atm_strikes


,expiry,strike,futures_price
8,FEB-2026,1120,1118.0
341,JAN-2026,1110,1111.0
5611,MAR-2026,1140,1125.0


In [69]:
atm_strikes = atm_strikes.reset_index(drop=True)

In [71]:
import os

os.makedirs("../data/clean", exist_ok=True)

In [73]:
options_df_clean.to_csv("../data/clean/options_clean.csv", index=False)
futures_df_clean.to_csv("../data/clean/futures_clean.csv", index=False)
atm_strikes.to_csv("../data/clean/atm_strikes.csv", index=False)

In [75]:
import os
os.listdir("../data/clean")


['.ipynb_checkpoints',
 'atm_strikes.csv',
 'futures_clean.csv',
 'nifty_features_5min.csv',
 'nifty_merged_5min.parquet',
 'options_clean.csv',
 'spot_clean.csv']

In [77]:
spot_df = pd.read_csv(r"C:\Users\Janvi\Quant_Trading_Assignment\data\nifty spot 5min.csv")

In [79]:
spot_df_clean = (
    spot_df
    .dropna(subset=["close"])
    .reset_index(drop=True)
)

In [81]:
spot_df_clean.head()

,timestamp,open,high,low,close,volume
0,2026-01-14,259.490,261.82,256.71,259.96,40019421
1,2026-01-13,258.720,261.81,258.39,261.05,45730847
2,2026-01-12,259.160,261.30,256.80,260.25,45263767
3,2026-01-09,259.075,260.21,256.22,259.37,39996967
4,2026-01-08,257.020,259.29,255.70,259.04,50419337


In [83]:
import os
os.makedirs("../data/clean", exist_ok=True)

spot_df_clean.to_csv("../data/clean/spot_clean.csv", index=False)


In [85]:
spot_df = pd.read_csv("../data/clean/spot_clean.csv")
futures_df = pd.read_csv("../data/clean/futures_clean.csv")
options_df = pd.read_csv("../data/clean/options_clean.csv")

In [87]:
print("Spot columns:", spot_df.columns)
print("Futures columns:", futures_df.columns)
print("Options columns:", options_df.columns)



Spot columns: Index(['timestamp', 'open', 'high', 'low', 'close', 'volume'], dtype='object')
Futures columns: Index(['CONTRACT_D', 'PREVIOUS_S', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRIC', 'SETTLEMENT', 'NET_CHANGE',
       'OI_NO_CON', 'TRADED_QUA', 'TRD_NO_CON', 'TRADED_VAL', 'instrument', 'expiry', 'option_type', 'strike'],
      dtype='object')
Options columns: Index(['CONTRACT_D', 'PREVIOUS_S', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'CLOSE_PRIC', 'SETTLEMENT', 'NET_CHANGE',
       'OI_NO_CON', 'TRADED_QUA', 'TRD_NO_CON', 'UNDRLNG_ST', 'NOTIONAL_V', 'PREMIUM_TR', 'instrument', 'expiry',
       'option_type', 'strike', 'futures_price', 'strike_diff'],
      dtype='object')


In [89]:
spot_df["timestamp"] = pd.to_datetime(spot_df["timestamp"])

In [91]:
futures_df["expiry"] = pd.to_datetime(futures_df["expiry"], format="%b-%Y")
options_df["expiry"] = pd.to_datetime(options_df["expiry"], format="%b-%Y")

In [93]:
spot_df["expiry"] = spot_df["timestamp"].dt.to_period("M").dt.to_timestamp()

merged_sf = spot_df.merge(
    futures_df,
    on="expiry",
    how="left",
    suffixes=("_spot", "_fut")
)

In [95]:
final_merged = merged_sf.merge(
    options_df,
    on="expiry",
    how="left",
    suffixes=("", "_opt")
)

MemoryError: Unable to allocate 1.52 GiB for an array with shape (13, 15687145) and data type float64

In [ ]:
final_merged.to_csv("../data/clean/nifty_merged_5min.csv", index=False)

In [ ]:
import pandas as pd

df = pd.read_csv("nifty_merged_5min.csv")

df.to_parquet(
    "nifty_merged_5min.parquet",
    engine="pyarrow",
    compression="snappy"
)

del df


In [ ]:
import os
os.getcwd()


In [ ]:
os.listdir()


In [96]:
import pandas as pd

csv_path = "../data/clean/nifty_merged_5min.csv"
parquet_path = "../data/clean/nifty_merged_5min.parquet"

chunks = pd.read_csv(csv_path, chunksize=200_000)

first = True
for chunk in chunks:
    chunk.to_parquet(
        parquet_path,
        engine="pyarrow",
        compression="snappy",
        index=False,
        append=not first
    )
    first = False

print("Parquet created successfully")


FileNotFoundError: [Errno 2] No such file or directory: '../data/clean/nifty_merged_5min.csv'

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

csv_path = "../data/clean/nifty_merged_5min.csv"
parquet_path = "../data/clean/nifty_merged_5min.parquet"

chunksize = 200_000
writer = None

for chunk in pd.read_csv(csv_path, chunksize=chunksize):
    table = pa.Table.from_pandas(chunk, preserve_index=False)

    if writer is None:
        writer = pq.ParquetWriter(parquet_path, table.schema, compression="snappy")

    writer.write_table(table)

if writer:
    writer.close()

print("Parquet created successfully")


In [99]:
import os

os.path.exists("../data/clean/nifty_merged_5min.parquet"), \
os.path.getsize("../data/clean/nifty_merged_5min.parquet") / (1024**2)


(True, 353.27867221832275)